### PCA Metric (Feature) Similarity Analysis 

Trying to understand how intercorrelated the metrics are via computing the PCA dim. reduction of the data

In [2]:
import pickle 
from utils import MODEL_SIZES, CHECKPOINT_STEPS, METRICS, sort_and_filter_metrics

import numpy as np 

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import scipy.stats
from scipy.stats import linregress

In [5]:
for model_size in MODEL_SIZES:
        
    # Setting up a matrix of the features
    X_avg = np.zeros((len(CHECKPOINT_STEPS), len(METRICS)))
    X = []

    for metric_idx, metric_name in enumerate(METRICS):
        with open(f'/home/rd654/pretraining-playground/computed_statistics/{model_size}/{metric_name}_per_layer.pkl', 'rb') as f:
            _metrics = pickle.load(f)
            # filter out only attention.dense and dense_4h_to_h layers 
            metrics = sort_and_filter_metrics(_metrics, filter_layer_name="ov_circuit", remove_heads=True)
        
        # NOTE: I'm not sure if I should do the SVD analysis on the average of the metrics or the individual metrics
        # maybe we just do both 
        
        X_curr = np.zeros((len(CHECKPOINT_STEPS), len(metrics)))

        layer_average_metric = None

        for layer_idx, (layer_name, layer_checkpoint_metrics) in enumerate(metrics.items()):
            if layer_average_metric is None:
                layer_average_metric = np.zeros(len(layer_checkpoint_metrics))
            
            layer_average_metric += np.array(layer_checkpoint_metrics)
 
            X_curr[:, layer_idx] = layer_checkpoint_metrics

        layer_average_metric /= len(metrics)

        X_avg[:, metric_idx] = layer_average_metric
        X.append(X_curr)

    scaler = StandardScaler()
    X_avg_scaled = scaler.fit_transform(X_avg)
    scaler = StandardScaler()
    # concatenate column wise X 
    X = np.concatenate(X, axis=1)

    #print out shape of X 

    X_scaled = scaler.fit_transform(X)

    print("---")
    print("Model Size: ", model_size)

    print("FOR AVERAGE METRICS")
    pca = PCA()
    pca.fit_transform(X_avg_scaled)
    # print("Principal coponents", pca.components_)
    print("Explained variance ratio", pca.explained_variance_ratio_)


    print("FOR INDIVIDUAL METRICS")
    pca = PCA()
    pca.fit_transform(X_scaled)
    # print("Principal components", pca.components_)
    print("Explained variance ratio", pca.explained_variance_ratio_)
    print("---")



---
Model Size:  70m
FOR AVERAGE METRICS
Explained variance ratio [0.74326579 0.13973215 0.10877259 0.00822947]
FOR INDIVIDUAL METRICS
Explained variance ratio [7.08622485e-01 1.46866678e-01 1.13558965e-01 1.02518490e-02
 7.01739349e-03 4.86872938e-03 3.65475421e-03 1.86811918e-03
 1.29559746e-03 8.60086444e-04 5.41919248e-04 1.96732854e-04
 1.43106950e-04 1.15740409e-04 4.95495155e-05 3.76378357e-05
 2.47386013e-05 1.68063964e-05 5.85134269e-06 1.53380610e-06
 1.28544475e-06 4.16846700e-07 2.25004174e-08 5.75614632e-10]
---
---
Model Size:  160m
FOR AVERAGE METRICS
Explained variance ratio [0.73922439 0.13563861 0.1068667  0.0182703 ]
FOR INDIVIDUAL METRICS
Explained variance ratio [7.06977355e-01 1.38819092e-01 1.08576694e-01 1.82107612e-02
 1.32203830e-02 4.74010667e-03 3.79002244e-03 1.79039952e-03
 1.32288448e-03 7.61404354e-04 5.62349287e-04 5.10667927e-04
 2.24490276e-04 1.55922088e-04 1.27314264e-04 6.83556654e-05
 6.11621300e-05 3.03440453e-05 1.94205109e-05 1.30458321e-05
 9.

### Linear Correlation between metric features 

Computing pair-wise linear correlation between features: cka scores and grad sim 

In [8]:
for model_size in MODEL_SIZES:

    # NOTE: computing the linear correlation between the cka_scores and grad_sim

    cka_scores, grad_sim = None, None

    for metric_name in ["cka_scores", "grad_sim"]:
        with open(f'/home/rd654/pretraining-playground/computed_statistics/{model_size}/{metric_name}_per_layer.pkl', 'rb') as f:
            _metrics = pickle.load(f)
            # filter out only attention.dense and dense_4h_to_h layers 
            metrics = sort_and_filter_metrics(_metrics, filter_layer_name="ov_circuit", remove_heads=True)

        if metric_name == "cka_scores":
            cka_scores = metrics
        else:
            grad_sim = metrics
   
    avg_corr_coeff = 0
    avg_r2 = 0

    for layer in cka_scores.keys():

        corr_coeff = scipy.stats.pearsonr(cka_scores[layer], grad_sim[layer])[0]
        slope, intercept, r_value, p_value, std_err = linregress(cka_scores[layer], grad_sim[layer])

        avg_corr_coeff += corr_coeff
        avg_r2 += r_value**2

    avg_corr_coeff /= len(cka_scores)
    avg_r2 /= len(cka_scores)
    
    print("Model Size: ", model_size)
    print("Average Linear Correlation Coeff.: ", avg_corr_coeff)
    print("Average R2 Value: ", avg_r2)

Model Size:  70m
Average Linear Correlation Coeff.:  -0.6276919088769429
Average R2 Value:  0.4432594070524094
Model Size:  160m
Average Linear Correlation Coeff.:  -0.5753619207366213
Average R2 Value:  0.40801597560499075
Model Size:  410m
Average Linear Correlation Coeff.:  -0.6826211517859152
Average R2 Value:  0.5650402215908864
Model Size:  1.4b
Average Linear Correlation Coeff.:  -0.7623811199228662
Average R2 Value:  0.6225179999027586
Model Size:  2.8b
Average Linear Correlation Coeff.:  -0.8738087147348669
Average R2 Value:  0.7729885114202066
